# TP 1 - Partie 2 : Inférence de Trajectoires et Interpolation de McCann

## Objectifs

Dans ce notebook, vous allez :
1. Reconstruire des trajectoires en chaînant les couplages OT entre snapshots successifs
2. Implémenter l'interpolation de McCann (géodésiques de Wasserstein)
3. Comparer les distributions interpolées avec les vraies distributions intermédiaires
4. Mesurer l'erreur de reconstruction en fonction du paramètre $\varepsilon$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ot
from scipy.spatial.distance import cdist
from matplotlib.colors import LogNorm
import sys
sys.path.append('../src')
from simulation import simulate_sde

# Configuration matplotlib
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## 1. Rappel : Interpolation de McCann

### Définition

Étant données deux distributions $\mu_0$ et $\mu_1$, et un plan de transport optimal $\pi^*$, l'**interpolation de McCann** (ou **géodésique de Wasserstein**) au temps $t \in [0, 1]$ est définie par :

$$
\mu_t := \left[(1-t)X + tY\right]_\# \pi^*
$$

où $(X, Y) \sim \pi^*$ et $f_\#\mu$ désigne la mesure image.

### Propriété géodésique

Cette interpolation est une **géodésique à vitesse constante** pour la distance de Wasserstein-2 :

$$
W_2(\mu_s, \mu_t) = |t - s| \cdot W_2(\mu_0, \mu_1)
$$

### Interprétation

Pour l'inférence de trajectoires, si on a calculé le couplage optimal entre deux snapshots, l'interpolation de McCann nous donne une prédiction naturelle de la distribution intermédiaire.

## 2. Chargement des données

On va générer deux ensembles de données :
1. **Snapshots espacés** : pour calculer les couplages OT
2. **Snapshots denses** : pour avoir la "vraie" distribution intermédiaire à comparer

In [ ]:
# Paramètres de simulation
n_particles = 1000
dim = 2
t0, t1 = 0.0, 1.0
dt = 1e-3
sigma = 0.5
seed = 42

# Snapshots espacés (pour calculer OT)
n_snapshots_sparse = 6
snapshot_times_sparse = np.linspace(t0, t1, n_snapshots_sparse)

snapshots_sparse = simulate_sde(
    n_particles=n_particles,
    dim=dim,
    t0=t0,
    t1=t1,
    dt=dt,
    sigma=sigma,
    snapshot_times=snapshot_times_sparse,
    potential_type='complex',
    seed=seed
)

# Snapshots denses (pour avoir les vraies distributions intermédiaires)
n_snapshots_dense = 21
snapshot_times_dense = np.linspace(t0, t1, n_snapshots_dense)

snapshots_dense = simulate_sde(
    n_particles=n_particles,
    dim=dim,
    t0=t0,
    t1=t1,
    dt=dt,
    sigma=sigma,
    snapshot_times=snapshot_times_dense,
    potential_type='complex',
    seed=seed
)

print(f"Snapshots espacés: {len(snapshots_sparse)} aux temps {snapshot_times_sparse}")
print(f"Snapshots denses: {len(snapshots_dense)} snapshots")

## 3. Calcul des couplages entre snapshots consécutifs

On va d'abord calculer les couplages OT entropique entre chaque paire de snapshots consécutifs.

In [ ]:
def compute_ot_coupling(X_source, X_target, epsilon):
    """
    Calcule le couplage OT entropique entre deux distributions empiriques.
    
    Parameters
    ----------
    X_source : ndarray, shape (n_source, d)
        Particules sources
    X_target : ndarray, shape (n_target, d)
        Particules cibles
    epsilon : float
        Paramètre de régularisation entropique
    
    Returns
    -------
    gamma : ndarray, shape (n_source, n_target)
        Plan de transport optimal
    """
    # TODO: Implémenter cette fonction
    # Hint: Créer les distributions uniformes a et b
    # Hint: Calculer la matrice de coût C
    # Hint: Utiliser ot.sinkhorn
    
    n_source = len(X_source)
    n_target = len(X_target)
    
    a = # VOTRE CODE ICI
    b = # VOTRE CODE ICI
    
    C = # VOTRE CODE ICI
    
    gamma = # VOTRE CODE ICI
    
    return gamma

# Calculer tous les couplages
times_sparse = sorted(snapshots_sparse.keys())
epsilon_theory = sigma * (times_sparse[1] - times_sparse[0])

print(f"Paramètre de régularisation (théorique): ε = {epsilon_theory:.4f}")
print("\nCalcul des couplages...")

couplings = {}
for i in range(len(times_sparse) - 1):
    t_start = times_sparse[i]
    t_end = times_sparse[i + 1]
    
    X_start = snapshots_sparse[t_start]
    X_end = snapshots_sparse[t_end]
    
    gamma = compute_ot_coupling(X_start, X_end, epsilon_theory)
    couplings[(t_start, t_end)] = gamma
    
    print(f"  [{t_start:.2f} → {t_end:.2f}] : couplage de forme {gamma.shape}")

print("\n✓ Couplages calculés")

## 4. Inférence de trajectoires par chaînage de couplages

Une fois qu'on a les couplages, on peut reconstruire des trajectoires en les chaînant. Pour chaque particule au temps $t_0$, on peut :
1. La "transporter" au temps $t_1$ selon $\gamma^*_{t_0, t_1}$
2. Puis la transporter au temps $t_2$ selon $\gamma^*_{t_1, t_2}$
3. Etc.

En pratique, pour une particule $i$ au temps $t_k$, on échantillonne son image au temps $t_{k+1}$ selon la distribution $\gamma^*_{t_k, t_{k+1}}(i, \cdot)$.

In [ ]:
def build_trajectories(snapshots_dict, couplings, n_trajectories=100, seed=42):
    """
    Construit des trajectoires en chaînant les couplages OT.
    
    Parameters
    ----------
    snapshots_dict : dict
        Dictionnaire {temps: array de particules}
    couplings : dict
        Dictionnaire {(t_start, t_end): matrice de couplage}
    n_trajectories : int
        Nombre de trajectoires à construire
    
    Returns
    -------
    trajectories : list of list of ndarray
        Liste de trajectoires, chaque trajectoire est une liste de positions
    """
    # TODO: Implémenter le chaînage de couplages
    # Hint: Partir de particules aléatoires au temps initial
    # Hint: Pour chaque temps, échantillonner la particule suivante selon gamma[current_idx, :]
    # Hint: Normaliser la distribution conditionnelle avant d'échantillonner
    
    rng = np.random.default_rng(seed)
    times = sorted(snapshots_dict.keys())
    
    # VOTRE CODE ICI
    trajectories = []
    
    return trajectories, times

# Construire les trajectoires
trajectories, times_traj = build_trajectories(
    snapshots_sparse, 
    couplings, 
    n_trajectories=50,
    seed=42
)

print(f"Construit {len(trajectories)} trajectoires")
if len(trajectories) > 0:
    print(f"Chaque trajectoire a {len(trajectories[0])} points")

In [ ]:
# Visualisation des trajectoires
fig, ax = plt.subplots(figsize=(10, 8))

# Tracer les trajectoires
for traj in trajectories:
    traj_array = np.array(traj)
    ax.plot(traj_array[:, 0], traj_array[:, 1], 'o-', alpha=0.3, markersize=4, linewidth=1)

# Marquer les snapshots
colors = plt.cm.viridis(np.linspace(0, 1, len(times_traj)))
for idx, t in enumerate(times_traj):
    X = snapshots_sparse[t]
    ax.scatter(X[:, 0], X[:, 1], c=[colors[idx]], s=5, alpha=0.1, 
              label=f't={t:.2f}' if idx in [0, len(times_traj)//2, len(times_traj)-1] else '')

ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.set_title('Trajectoires inférées par chaînage de couplages OT')
ax.legend()
ax.axis('equal')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Interpolation de McCann

### 5.1 Implémentation

Pour calculer l'interpolation de McCann entre deux distributions $\mu_0$ et $\mu_1$ avec un couplage $\pi^*$, on doit :

1. Échantillonner des paires $(x, y) \sim \pi^*$
2. Pour chaque paire, calculer le point interpolé : $z_t = (1-t)x + ty$
3. La distribution empirique de tous les $z_t$ est notre $\mu_t$ interpolée

In [ ]:
def mccann_interpolation(X_source, X_target, gamma, t, n_samples=1000, seed=42):
    """
    Calcule l'interpolation de McCann au temps t entre deux distributions.
    
    Parameters
    ----------
    X_source : ndarray, shape (n_source, d)
        Particules sources (au temps 0)
    X_target : ndarray, shape (n_target, d)
        Particules cibles (au temps 1)
    gamma : ndarray, shape (n_source, n_target)
        Plan de transport optimal
    t : float
        Temps d'interpolation (entre 0 et 1)
    n_samples : int
        Nombre d'échantillons à générer
    
    Returns
    -------
    X_interp : ndarray, shape (n_samples, d)
        Particules interpolées au temps t
    """
    # TODO: Implémenter l'interpolation de McCann
    # Hint: Aplatir gamma pour avoir une distribution sur les paires (i,j)
    # Hint: Échantillonner n_samples paires selon gamma_flat
    # Hint: Convertir les indices plats en indices (i, j)
    # Hint: Calculer (1-t) * X_source[i] + t * X_target[j]
    
    rng = np.random.default_rng(seed)
    
    n_source, n_target = gamma.shape
    
    # VOTRE CODE ICI
    X_interp = None
    
    return X_interp

### 5.2 Test sur un intervalle

Testons l'interpolation de McCann entre les deux premiers snapshots.

In [ ]:
# Prendre les deux premiers snapshots
t_start = times_sparse[0]
t_end = times_sparse[1]
X_start = snapshots_sparse[t_start]
X_end = snapshots_sparse[t_end]
gamma_01 = couplings[(t_start, t_end)]

# Calculer plusieurs interpolations
t_interp_values = [0.25, 0.5, 0.75]

fig, axes = plt.subplots(1, 5, figsize=(20, 4))

# Distribution initiale
axes[0].scatter(X_start[:, 0], X_start[:, 1], alpha=0.5, s=10)
axes[0].set_title(f't = {t_start:.2f} (source)')
axes[0].set_xlabel('$x_0$')
axes[0].set_ylabel('$x_1$')
axes[0].axis('equal')
axes[0].set_xlim(-5, 5)
axes[0].set_ylim(-4, 4)
axes[0].grid(True, alpha=0.3)

# Interpolations
for idx, t_frac in enumerate(t_interp_values):
    X_interp = mccann_interpolation(X_start, X_end, gamma_01, t_frac, n_samples=1000)
    
    if X_interp is not None:
        axes[idx + 1].scatter(X_interp[:, 0], X_interp[:, 1], alpha=0.5, s=10, color='orange')
        t_actual = t_start + t_frac * (t_end - t_start)
        axes[idx + 1].set_title(f't = {t_actual:.2f} (interpolé, α={t_frac})')
        axes[idx + 1].set_xlabel('$x_0$')
        axes[idx + 1].axis('equal')
        axes[idx + 1].set_xlim(-5, 5)
        axes[idx + 1].set_ylim(-4, 4)
        axes[idx + 1].grid(True, alpha=0.3)

# Distribution finale
axes[4].scatter(X_end[:, 0], X_end[:, 1], alpha=0.5, s=10, color='red')
axes[4].set_title(f't = {t_end:.2f} (cible)')
axes[4].set_xlabel('$x_0$')
axes[4].axis('equal')
axes[4].set_xlim(-5, 5)
axes[4].set_ylim(-4, 4)
axes[4].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Comparaison avec les vraies distributions intermédiaires

Maintenant, comparons les distributions interpolées avec McCann aux **vraies** distributions intermédiaires qu'on a simulées.

### 6.1 Mesure de distance : Wasserstein-2

Pour quantifier l'écart entre deux distributions empiriques, on va utiliser la distance de Wasserstein-2 (calculée avec OT **non régularisé** cette fois).

In [ ]:
def wasserstein2_distance(X, Y):
    """
    Calcule la distance de Wasserstein-2 entre deux distributions empiriques.
    
    Utilise l'OT exact (EMD) pour une mesure précise.
    """
    # TODO: Implémenter le calcul de W2
    # Hint: Créer distributions uniformes a, b
    # Hint: Calculer matrice de coût C avec 'sqeuclidean'
    # Hint: Utiliser ot.emd2 et prendre la racine carrée
    # Note: Si EMD échoue, utiliser Sinkhorn avec epsilon petit (1e-3)
    
    n, m = len(X), len(Y)
    
    # VOTRE CODE ICI
    
    return 0.0  # Remplacer par le vrai calcul

### 6.2 Calcul des erreurs d'interpolation

Pour chaque paire de snapshots consécutifs, on va :
1. Calculer plusieurs interpolations de McCann à différents temps intermédiaires
2. Récupérer les vraies distributions à ces mêmes temps (depuis `snapshots_dense`)
3. Mesurer la distance de Wasserstein-2 entre interpolation et vérité terrain

In [ ]:
# On va tester différentes valeurs d'epsilon
epsilon_values = [
    epsilon_theory / 5,
    epsilon_theory,
    epsilon_theory * 5
]

print("Calcul des erreurs d'interpolation pour différents ε...\n")

# TODO: Calculer les erreurs pour chaque epsilon
# Hint: Pour chaque intervalle [t_start, t_end]
# Hint: Pour chaque epsilon, calculer le couplage
# Hint: Pour chaque temps intermédiaire, calculer l'interpolation de McCann
# Hint: Comparer avec la vraie distribution (snapshots_dense)
# Hint: Mesurer avec wasserstein2_distance

results = {eps: {'times': [], 'errors': []} for eps in epsilon_values}

# VOTRE CODE ICI

print("\n✓ Évaluation terminée")

### 6.3 Visualisation des erreurs

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['blue', 'green', 'red']
labels = [f'ε = {eps:.6f}' + (' (théorique)' if np.isclose(eps, epsilon_theory) else '') 
          for eps in epsilon_values]

for eps, color, label in zip(epsilon_values, colors, labels):
    times = results[eps]['times']
    errors = results[eps]['errors']
    
    if len(times) > 0:
        ax.plot(times, errors, 'o-', color=color, label=label, markersize=6, linewidth=2)

# Marquer les temps des snapshots espacés
for t in times_sparse:
    ax.axvline(t, color='gray', linestyle='--', alpha=0.3, linewidth=1)

ax.set_xlabel('Temps t')
ax.set_ylabel('Erreur W₂(interpolation, vérité terrain)')
ax.set_title('Erreur d\'interpolation de McCann vs temps\n(comparaison avec distribution simulée)')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Statistiques
print("\nStatistiques des erreurs :")
print("="*70)
for eps in epsilon_values:
    errors = results[eps]['errors']
    if len(errors) > 0:
        print(f"ε = {eps:.6f}:")
        print(f"  Erreur moyenne : {np.mean(errors):.4f}")
        print(f"  Erreur médiane : {np.median(errors):.4f}")
        print(f"  Erreur max     : {np.max(errors):.4f}")
        print()

## 7. Questions de réflexion

### Question 1
Quelle valeur de $\varepsilon$ donne la meilleure reconstruction ? Est-ce cohérent avec la théorie ?

### Question 2
Comment évolue l'erreur au cours du temps ? Y a-t-il des régions temporelles où l'interpolation fonctionne mieux ?

### Question 3
Que se passe-t-il visuellement quand $\varepsilon$ est trop petit ou trop grand ?

### Question 4
Observez-vous un pic d'erreur autour de t ≈ 0.5-0.6 ? Pourquoi ?
(Hint: C'est le moment du branchement dans le potentiel complexe)

## 8. Conclusion

Dans ce notebook, vous avez :

✅ Reconstruit des trajectoires par chaînage de couplages OT  
✅ Implémenté l'interpolation de McCann  
✅ Comparé les interpolations avec les vraies distributions intermédiaires  
✅ Mesuré l'erreur de reconstruction via la distance de Wasserstein-2  

### Points clés à retenir

1. **Interpolation de McCann** : Fournit une façon naturelle et géométriquement fondée d'interpoler entre distributions

2. **Choix de $\varepsilon$** : Le paramètre $\varepsilon = \sigma \Delta t$ n'est pas arbitraire - il découle de la connection avec le problème de Schrödinger

3. **Erreur d'interpolation** : Même avec le bon $\varepsilon$, l'interpolation n'est pas parfaite car le processus sous-jacent n'est pas exactement un transport optimal

4. **Branchement** : L'erreur est particulièrement élevée aux points de branchement, ce qui montre les limitations de l'approche

### Limitations et perspectives

- **Limitation 1** : On a supposé qu'il n'y a pas de branchement (cellules qui se divisent) - mais notre potentiel complexe montre que c'est difficile !
- **Limitation 2** : Les distributions observées sont bruitées (peu d'échantillons)

→ **Session 2 du TP** : On verra comment optimiser simultanément sur les marginales pour gérer le bruit (gWOT, Chizat et al. 2022)